In [2]:
%load_ext autoreload
%autoreload 2

# load embeddings
from data.embeddings import load_saved_embeddings

train_embeddings = load_saved_embeddings(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    split="train",
    pooler="mean_with_attention",
    layer=24
)

eval_embeddings = load_saved_embeddings(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    split="eval",
    pooler="mean_with_attention",
    layer=24
)

test_embeddings = load_saved_embeddings(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    split="test",
    pooler="mean_with_attention",
    layer=24
)

# load classifier 
from data.models import load_saved_wrapperbox_model


knn_clf = load_saved_wrapperbox_model(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    pooler="mean_with_attention",
    wrapperbox="KNN"
)

svm_clf = load_saved_wrapperbox_model(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    pooler="mean_with_attention",
    wrapperbox="SVM",
)

dt_clf = load_saved_wrapperbox_model(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    pooler="mean_with_attention",
    wrapperbox="DecisionTree",
)

lmeans_clf = load_saved_wrapperbox_model(
    dataset="esnli",
    model="deberta-large",
    seed=42,
    pooler="mean_with_attention",
    wrapperbox="LMeans",
)

# load labels 
from data.datasets import load_dataset_from_hf, load_labels_at_split
import numpy as np
dataset = load_dataset_from_hf(dataset="esnli")
train_labels = load_labels_at_split(dataset, "train")
eval_labels = load_labels_at_split(dataset, "eval")
train_eval_embeddings = np.vstack([train_embeddings, eval_embeddings])
train_eval_labels = np.concatenate([train_labels, eval_labels])
test_labels = load_labels_at_split(dataset, "test")

from datasets import DatasetDict, concatenate_datasets
train_eval_dataset = concatenate_datasets([dataset["train"], dataset["eval"]])
dataset_dict = DatasetDict(
    {"train": train_eval_dataset, "test": dataset["test"]}
)

/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.1.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.1.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/samsoup/.cache/huggingface/token
Login successful


In [3]:
from pprint import pprint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from utils.inference import compute_metrics

l2 = 500
logit_clf = LogisticRegression(penalty="l2", C= 1 / l2)

logit_clf.fit(train_eval_embeddings, train_eval_labels)

predictions = logit_clf.predict(test_embeddings)

# Print some metrics
testset_perfm = compute_metrics(
    y_true=test_labels, y_pred=predictions, is_multiclass=True, prefix="test"
)
pprint(testset_perfm)
print(classification_report(y_true=test_labels, y_pred=predictions))

{'test_0_f1': 0.9241727259237276,
 'test_0_precision': 0.92376149510531,
 'test_0_recall': 0.9245843230403801,
 'test_1_f1': 0.8861586314152411,
 'test_1_precision': 0.8872625350358144,
 'test_1_recall': 0.8850574712643678,
 'test_2_f1': 0.9498379379533879,
 'test_2_precision': 0.9491054904380012,
 'test_2_recall': 0.950571516836577,
 'test_accuracy': 0.9201954397394136,
 'test_f1': 0.9200564317641189,
 'test_macro_f1': 0.9200564317641189,
 'test_macro_precision': 0.9200431735263752,
 'test_macro_recall': 0.920071103713775,
 'test_micro_f1': 0.9201954397394136,
 'test_micro_precision': 0.9201954397394136,
 'test_micro_recall': 0.9201954397394136,
 'test_precision': 0.9200431735263752,
 'test_recall': 0.920071103713775,
 'test_weighted_f1': 0.9201734304348423,
 'test_weighted_precision': 0.9201528184388009,
 'test_weighted_recall': 0.9201954397394136}
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3368
           1       0.89     

/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
